In [4]:
# pip install dash
# pip install plotly

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\pensh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [13]:
import pandas as pd
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px

app = Dash(__name__)

df = pd.read_csv('transfers.txt').iloc[: , 1:]
df = df[df['From'] != 'College']

app.layout = html.Div(children = [
    html.H1(children='NFL Transactions', style={'textAlign':'center', 'color':'white'}),
    html.H2(children='Updated 2023 Week 2', style={'textAlign':'center', 'color':'white'}),


    html.Div(children=[
        html.Div(children=[
            html.Label('From Team Select', style={'color':'white'}),
            dcc.Dropdown((['All Teams'] + sorted(df.From.unique())), '49ers', id='from-dropdown'),
        ], style={'padding': 10, 'flex':1}),

        html.Div(children=[
            html.Label('To Team Select', style={'color':'white'}),
            dcc.Dropdown((['All Teams'] + sorted(df.From.unique())), 'All Teams', id='to-dropdown'),
        ], style={'padding': 10, 'flex': 1}),
    ], style={'padding': 10, 'display': 'flex', 'flexDirection': 'row', 'scale': 2}),


    html.Div(children=[
        html.Div(children=[
            html.Div(children=[
                html.Label('Include From: College?', style={'color':'white'}),
                dcc.RadioItems(['No', 'Yes (Load Time Warning)', 'Individual (Severe Load Time Warning)'], 'No', style={'color':'white'}, id = 'collegeCheck'),
            ], style={'padding': 10,}),

            html.Div(children=[
                html.Label('Immaculate Grid Mode?', style={'color':'white'}),
                dcc.RadioItems(['No', 'Yes'], 'No', style={'color':'white'}, id = 'immaculateCheck'),
            ], style={'padding': 10,}),
        ], style={'padding': 5, 'display': 'flex', 'flexDirection': 'column'}),

        html.Div(children=[
            html.Br(),
            html.Label('From Year', style={'color':'white'}),
            dcc.Slider(min=2000, max=2024,
                marks={str(i) : {'label' : str(i), 'style':{'color':'white'}} for i in range(2000, 2025, 2)}, value=2020,
                id='from-year-slider'),

            html.Br(),
            html.Label('To Year', style={'color':'white'}),
            dcc.Slider(min=2000, max=2024,
                marks={str(i) : {'label' : str(i), 'style':{'color':'white'}} for i in range(2000, 2025, 2)}, value=2024,
                id='to-year-slider'),
        ], style={'padding': 10, 'display': 'flex', 'flexDirection': 'column', 'flex': 1, 'scale': 2}),
    ], style={'padding': 10, 'display': 'flex', 'flexDirection': 'row', 'scale': 2}),

    dcc.Graph(id='graph-content')
], style={'backgroundColor':'black', 'scale': 2})


@callback(
    Output('graph-content', 'figure'),
    Input('from-dropdown', 'value'),
    Input('from-year-slider', 'value'),
    Input('to-dropdown', 'value'),
    Input('to-year-slider', 'value'),
    Input('collegeCheck', 'value'),
    Input('immaculateCheck', 'value'),
)

def update_graph(FromTeam, FromYear, ToTeam, ToYear, College, Immaculate):
    data = pd.read_csv('transfers.txt').iloc[: , 1:]
    if College == 'No':
        data = data[data['From'] != 'College']
    elif College == 'Individual (Severe Load Time Warning)':
        data.loc[data.From == 'College', 'From'] = data['College']
        data.loc[data.From.str.contains(','), 'From'] = data['From'].str.split(',')
        data['From'] = data['From'].apply(lambda x: x[-1] if isinstance(x, list) else x)

    if Immaculate == 'Yes':
        From = data[['BirthDate', 'Player', 'From', 'Transactions']].rename(columns = {'From' : 'Team'})
        To = data[['BirthDate', 'Player', 'To', 'Transactions']].rename(columns = {'To' : 'Team'})
        data = pd.merge(From, To, on = ['BirthDate', 'Player'], how = "right").drop(columns = ['Transactions_y']).rename(columns = {'Team_x' : 'Team1', 'Team_y' : 'Team2','Transactions_x' : 'Transactions'}).sort_values(by = ['Player', 'Team1', 'Team2'], ascending = True).drop_duplicates(subset = ['BirthDate', 'Player', 'Team1', 'Team2'], keep = 'last').reset_index(drop=True)
        temp = pd.DataFrame(data).rename(columns = {'Team2' : 'Team1', 'Team1' : 'Team2'})
        data = pd.concat([data, temp]).sort_values(by = ['Player', 'Team1', 'Team2'], ascending = True).drop_duplicates(subset = ['BirthDate', 'Player', 'Team1', 'Team2'], keep = 'last').reset_index(drop=True)
        data = data[data.Team1 != data.Team2]

        if (FromTeam == 'All Teams') & (ToTeam == 'All Teams'):
            return (px.treemap(data, path=[px.Constant('Immaculate Grid!'), 'Team1', 'Team2', 'Player'], values = 'Transactions', template = 'plotly_dark'))
        elif FromTeam == 'All Teams': 
            data = data[data.Team2==ToTeam]
            return (px.treemap(data, path=[px.Constant("Immaculate Grid! (" + str(ToTeam) + ")"), 'Team1', 'Team2', 'Player'], values = 'Transactions', template = 'plotly_dark'))
        elif ToTeam == 'All Teams': 
            data = data[data.Team1==FromTeam]
            return (px.treemap(data, path=[px.Constant("Immaculate Grid! (" + str(FromTeam) + ")"), 'Team1', 'Team2', 'Player'], values = 'Transactions', template = 'plotly_dark'))
        else:
            data = data[data.Team2==ToTeam]
            data = data[data.Team1==FromTeam]
            return (px.treemap(data, path=[px.Constant("Immaculate Grid! (" + str(FromTeam) + ") -> (" + str(ToTeam) + ")"), 'Player'], values = 'Transactions', template = 'plotly_dark'))

    if (FromTeam == 'All Teams') & (ToTeam == 'All Teams'):
        data = data[data.Year>=FromYear]
        data = data[data.Year<=ToYear]
        return (px.treemap(data, path=[px.Constant('All Transactions'), 'From', 'To', 'Player'], values = 'Transactions', template = 'plotly_dark'))
    elif FromTeam == 'All Teams': 
        data = data[data.To==ToTeam]
        data = data[data.Year>=FromYear]
        data = data[data.Year<=ToYear]
        return (px.treemap(data, path=[px.Constant("To: " + str(ToTeam)), 'From', 'Player'], values = 'Transactions', template = 'plotly_dark'))
    elif ToTeam == 'All Teams': 
        data = data[data.From==FromTeam]  
        data = data[data.Year>=FromYear]
        data = data[data.Year<=ToYear]
        return (px.treemap(data, path=[px.Constant("From: " + str(FromTeam)), 'To', 'Player'], values = 'Transactions', template = 'plotly_dark'))
    else:
        data = data[data.To==ToTeam]
        data = data[data.From==FromTeam]
        data = data[data.Year>=FromYear]
        data = data[data.Year<=ToYear]
        return (px.treemap(data, path=[px.Constant("From: " + str(FromTeam) + " To: " + str(ToTeam)), 'Player'], values = 'Transactions', template = 'plotly_dark'))

if __name__ == '__main__':
    app.run(debug=True)